In [43]:
import numpy as np
import pandas as pd
import os
import wandb
from pprint import pprint

In [75]:
ablation_proj_name = 'REPORT_ABL_direct-opt'
ablation_dir = os.path.join("..", "..", "EXP_OUTPUTS", ablation_proj_name)

all_files = os.listdir(ablation_dir)
# Filter only directories
directories = [d for d in all_files if os.path.isdir(os.path.join(ablation_dir, d))]
# Filter out directories with the specified postfix
filtered_directories = sorted([d[:d.find("_run")] for d in directories])
# Get unique directory names
unique_directories = sorted(list(set(filtered_directories)))

unique_directories

['ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt0.0',
 'ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt2.0',
 'ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-neg-srt2.0',
 'ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg-srt0.0',
 'ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg-srt2.0',
 'ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non-neg-srt2.0']

In [80]:
ablation_mean_shading = {}
ablation_mean_render = {}

for albation_config in unique_directories:
    
    # Get the matching dirs
    matching_directories = sorted([d for d in all_files if d.startswith(albation_config) and os.path.isdir(os.path.join(ablation_dir, d))])
    print("Processing")
    pprint(matching_directories)
    print()
    
    for table_name in ['render', 'shading']:
        # Create an empty DataFrame to store the combined data
        combined_data = pd.DataFrame()

        # Iterate over each matching directory: get the average
        for directory in matching_directories:
            # Construct the path to the CSV file in the current directory
            csv_path = os.path.join(ablation_dir, directory, 'test_tables', f'table_avg_metrics_{table_name}.csv')
            

            # Read the CSV file into a DataFrame
            df = pd.read_csv(csv_path)

            # Append the DataFrame to the combined_data DataFrame
            combined_data = pd.concat([combined_data,df], ignore_index=True)


        mean_row = combined_data.mean()
        
        if table_name == 'render':
            ablation_mean_render[albation_config] = mean_row
        else:
            ablation_mean_shading[albation_config] = mean_row
        
    

Processing
['ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt0.0_run1_apricot-field-1',
 'ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt0.0_run2_dark-waterfall-2',
 'ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt0.0_run3_deft-lion-3',
 'ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt0.0_run4_peachy-river-4',
 'ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt0.0_run5_giddy-brook-5']

Processing
['ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt2.0_run1_glowing-bee-31',
 'ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt2.0_run2_sunny-breeze-32',
 'ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt2.0_run3_lyric-tree-33',
 'ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt2.0_run4_eager-cherry-34',
 'ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt2.0_run5_fiery-firebrand-35']

Processing
['ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-neg-srt2.0_run1_rich-meadow-11',
 'ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-neg-srt2.0_run2_wise-frost-12',
 'ablation_lr1.6e-

In [81]:
pprint(ablation_mean_shading)

{'ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt0.0': MSE           0.036851
LMSE          0.009817
LMSE_SLI2     3.033015
PSNR         14.851072
DSSIM         0.039757
SIL2          9.786200
dtype: float64,
 'ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt2.0': MSE           0.036305
LMSE          0.009742
LMSE_SLI2     2.083650
PSNR         14.876728
DSSIM         0.039231
SIL2          8.872779
dtype: float64,
 'ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-neg-srt2.0': MSE           0.031567
LMSE          0.008543
LMSE_SLI2     1.544746
PSNR         15.616729
DSSIM         0.038630
SIL2          9.075657
dtype: float64,
 'ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg-srt0.0': MSE           0.032959
LMSE          0.008777
LMSE_SLI2     2.154071
PSNR         15.307506
DSSIM         0.036152
SIL2          8.846092
dtype: float64,
 'ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg-srt2.0': MSE           0.032866
LMSE          0.008763
LMSE_SLI2     1.957949
PSNR         15.331

In [82]:
# Define a custom style to left-align the row index
styles = [
    dict(selector="th", props=[("text-align", "left")]),
    dict(selector="td", props=[("text-align", "left")]),
]


In [83]:
ablation_render_df = pd.DataFrame(ablation_mean_render).T
ablation_render_df.style.set_table_styles(styles)

,MSE,LMSE,LMSE_SLI2,PSNR,DSSIM,SIL2
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt0.0,0.027918,0.007190,4.151902,15.745746,0.055979,14.821936
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt2.0,0.026904,0.006923,3.503806,15.857938,0.056189,14.641187
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-neg-srt2.0,0.027829,0.007159,3.128340,15.682304,0.057890,16.618164
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg-srt0.0,0.026038,0.006684,3.491950,16.004069,0.054814,14.088887
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg-srt2.0,0.026147,0.006713,3.401816,15.983546,0.055624,14.813513
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non-neg-srt2.0,0.027602,0.007100,3.044776,15.719852,0.057788,16.731366


In [84]:
ablation_shading_df = pd.DataFrame(ablation_mean_shading).T
ablation_shading_df.style.set_table_styles(styles)

,MSE,LMSE,LMSE_SLI2,PSNR,DSSIM,SIL2
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt0.0,0.036851,0.009817,3.033015,14.851072,0.039757,9.786200
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-srt2.0,0.036305,0.009742,2.083650,14.876728,0.039231,8.872779
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-neg-srt2.0,0.031567,0.008543,1.544746,15.616729,0.038630,9.075657
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg-srt0.0,0.032959,0.008777,2.154071,15.307506,0.036152,8.846092
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg-srt2.0,0.032866,0.008763,1.957949,15.331908,0.036688,8.846820
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non-neg-srt2.0,0.031299,0.008465,1.410481,15.644726,0.038324,8.971629


In [85]:

#albation_table = pd.concat({'shading': pd.concat(ablation_render_df, keys='keys'), 'render': pd.concat(ablation_shading_df, keys='keys')}, axis=1)
combined_df = pd.concat([ablation_render_df, ablation_shading_df], keys=['Render', 'Shading'], axis=1, names=['Source', 'Metric'])
combined_df

Source                                                Render            \
Metric                                                   MSE      LMSE   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.027918  0.007190   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.026904  0.006923   
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-...  0.027829  0.007159   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  0.026038  0.006684   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  0.026147  0.006713   
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non...  0.027602  0.007100   

Source                                                                   \
Metric                                             LMSE_SLI2       PSNR   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  4.151902  15.745746   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  3.503806  15.857938   
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-...  3.128340  15.682304   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  3.491950  16.004069   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  3.401816  15.983546   
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non...  3.044776  15.719852   

Source                                                                   \
Metric                                                 DSSIM       SIL2   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.055979  14.821936   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.056189  14.641187   
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-...  0.057890  16.618164   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  0.054814  14.088887   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  0.055624  14.813513   
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non...  0.057788  16.731366   

Source                                               Shading            \
Metric                                                   MSE      LMSE   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.036851  0.009817   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.036305  0.009742   
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-...  0.031567  0.008543   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  0.032959  0.008777   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  0.032866  0.008763   
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non...  0.031299  0.008465   

Source                                                                   \
Metric                                             LMSE_SLI2       PSNR   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  3.033015  14.851072   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  2.083650  14.876728   
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-...  1.544746  15.616729   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  2.154071  15.307506   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  1.957949  15.331908   
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non...  1.410481  15.644726   

Source                                                                  
Metric                                                 DSSIM      SIL2  
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.039757  9.786200  
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.039231  8.872779  
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-...  0.038630  9.075657  
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  0.036152  8.846092  
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  0.036688  8.846820  
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non...  0.038324  8.971629

In [97]:
def format_values(value):
    if abs(value) < 1:
        return f"{value:.3g}"
    else:
        return f"{value:.2f}"
    
# def format_values(value):
#     if abs(value) < 1:
#         return series.apply(lambda x: f"{x:.2g}")
#     else:
#         return series.apply(lambda x: f"{value:.2f}")

In [98]:
combined_df.applymap(format_values)


/tmp/ipykernel_606619/4235678439.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined_df.applymap(format_values)


Source                                              Render                     \
Metric                                                 MSE     LMSE LMSE_SLI2   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.0279  0.00719      4.15   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.0269  0.00692      3.50   
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-...  0.0278  0.00716      3.13   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...   0.026  0.00668      3.49   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  0.0261  0.00671      3.40   
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non...  0.0276   0.0071      3.04   

Source                                                                    \
Metric                                               PSNR   DSSIM   SIL2   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  15.75   0.056  14.82   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  15.86  0.0562  14.64   
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-...  15.68  0.0579  16.62   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  16.00  0.0548  14.09   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  15.98  0.0556  14.81   
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non...  15.72  0.0578  16.73   

Source                                             Shading                     \
Metric                                                 MSE     LMSE LMSE_SLI2   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.0369  0.00982      3.03   
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  0.0363  0.00974      2.08   
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-...  0.0316  0.00854      1.54   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...   0.033  0.00878      2.15   
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  0.0329  0.00876      1.96   
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non...  0.0313  0.00846      1.41   

Source                                                                   
Metric                                               PSNR   DSSIM  SIL2  
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  14.85  0.0398  9.79  
ablation_lr1.6e-4_e3_sf2_init-rand_rgb_non-neg-...  14.88  0.0392  8.87  
ablation_lr1.6e-4_e3_sf2_init-rand_shading_non-...  15.62  0.0386  9.08  
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  15.31  0.0362  8.85  
ablation_lr2.5e-5_e3_sf2_init-zeros_rgb_non-neg...  15.33  0.0367  8.85  
ablation_lr2.5e-5_e3_sf2_init-zeros_shading_non...  15.64  0.0383  8.97

In [100]:
row_names = ["Init `rand`", "Init `rand` + Non-neg", "Init `rand` + Non-neg + Shading loss", 
            "Init `zeros`", "Init `zeros` + Non-neg", "Init `zeros` + Non-neg + Shading loss"]

In [103]:
combined_df.index = row_names


In [104]:
combined_df

Source                                   Render                      \
Metric                                      MSE      LMSE LMSE_SLI2   
Init `rand`                            0.027918  0.007190  4.151902   
Init `rand` + Non-neg                  0.026904  0.006923  3.503806   
Init `rand` + Non-neg + Shading loss   0.027829  0.007159  3.128340   
Init `zeros`                           0.026038  0.006684  3.491950   
Init `zeros` + Non-neg                 0.026147  0.006713  3.401816   
Init `zeros` + Non-neg + Shading loss  0.027602  0.007100  3.044776   

Source                                                                 \
Metric                                      PSNR     DSSIM       SIL2   
Init `rand`                            15.745746  0.055979  14.821936   
Init `rand` + Non-neg                  15.857938  0.056189  14.641187   
Init `rand` + Non-neg + Shading loss   15.682304  0.057890  16.618164   
Init `zeros`                           16.004069  0.054814  14.088887   
Init `zeros` + Non-neg                 15.983546  0.055624  14.813513   
Init `zeros` + Non-neg + Shading loss  15.719852  0.057788  16.731366   

Source                                  Shading                      \
Metric                                      MSE      LMSE LMSE_SLI2   
Init `rand`                            0.036851  0.009817  3.033015   
Init `rand` + Non-neg                  0.036305  0.009742  2.083650   
Init `rand` + Non-neg + Shading loss   0.031567  0.008543  1.544746   
Init `zeros`                           0.032959  0.008777  2.154071   
Init `zeros` + Non-neg                 0.032866  0.008763  1.957949   
Init `zeros` + Non-neg + Shading loss  0.031299  0.008465  1.410481   

Source                                                                
Metric                                      PSNR     DSSIM      SIL2  
Init `rand`                            14.851072  0.039757  9.786200  
Init `rand` + Non-neg                  14.876728  0.039231  8.872779  
Init `rand` + Non-neg + Shading loss   15.616729  0.038630  9.075657  
Init `zeros`                           15.307506  0.036152  8.846092  
Init `zeros` + Non-neg                 15.331908  0.036688  8.846820  
Init `zeros` + Non-neg + Shading loss  15.644726  0.038324  8.971629